In [32]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 7.10 콜백 함수에 추가적 상태 넣기
- 콜백 함수에 추가 상태를 넣고 내부 호출에 사용하고 싶을 때

In [18]:
def apply_async(func, args, *, callback):
    #결과 계산
    result = func(*args)
    
    #결과 값으로 콜백 함수 호출
    callback(result)

In [2]:
def print_result(result):
    print('Got:', result)
    
def add(x,y):
    return x+y

In [3]:
apply_async(add, (2, 3), callback=print_result)

Got: 5


In [4]:
apply_async(add, ('hello', 'world'), callback=print_result)

Got: helloworld


- print_result() 함수는 결과 값만 하나의 인자로 받음 -> 부족한 정보로 인해 문제 발생할 수 있

### 1. 바운드-메소드(bound-method) 사용

In [5]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    def handler(self, result):
        self.sequence += 1 #결과값을 받을 때마다 늘어나는 내부 시퀀스 숫자
        print('[{}] Got: {}'.format(self.sequence, result))

In [7]:
r = ResultHandler()
apply_async(add, (2, 3), callback = r.handler) #바운드 메소드 핸들러를 콜백으로 사용

[1] Got: 5


In [8]:
apply_async(add, ('hello', 'world'), callback=r.handler)

[2] Got: helloworld


- 아니면 클래스 대신 클로저를 사용해소 상태 저장해도 됨

In [10]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

In [11]:
handler = make_handler()
apply_async(add, (2, 3), callback = handler)

[1] Got: 5


In [12]:
apply_async(add, ('hello', 'world'), callback=handler)

[2] Got: helloworld


### 2. 코루틴(coroutine) 사용
- next() 를 호출해야 함
- w제너레이터에 데이터를 푸시하고 싶을 때 ( 외부에서 제공되는 데이터를 소비하는 역할을 하고 싶을 때

- 값을 받을 때 yield 키워드는 = 오른쪽에 있는 표현식으로 사용할 수 있음 
- 객체에 대해 send() 메서드를 사용해서 값을 함수로 다시 전달함

In [34]:
def printer():
    ## 1
    while True:
        line = (yield)  ## 2    #리턴이 아니라 입력의 목적
        print (line)
prn = printer() #printer() 함수 호출하여 코루틴 생성
# 생성 직후의 코루틴의 실행 위치는 #1

In [37]:
next(prn) # 2 지점까지 실행됨. 우변이 평가되는 시점에 코루틴이 한 번 멈춤.
# ********다음 동작으로 데이터를 반환하는 것이 아니라 데이터를 입력 받음***********
# 다시 실행될 때 외부에서 들여온 값으로 yield가 평가됨 -> 그 값이 line으로 들어감
prn.send(1) ## 1 값을 코루틴에 밀어넣음. print()문을 실행하고 다시 yield 위치까지 실행됨.

None
1


In [49]:
def test(i):
    print('start coroutine')
    while True:
        value = yield i
        i += value

b = test(5)
next(b) # print 출력 후 5 출력. yield i 부분까지 옴

start coroutine


5

In [50]:
b.send(3) #yield를 통해 3을 전달하여 value가 3이 됨. 이후 i += value 줄을 거쳐 i=8이 되고 한바퀴 돌아 8을 출력

8

In [51]:
b.send(5)

13

** 코루틴은 제너레이터와 달리 생성 후에 무조건 next()를 한 번 실행해서 값을 받을 수 있는 상태로 만들어 주어야 함.

In [48]:
def make_handler():
    sequence = 0
    while True:
        result = yield # 바인딩 구문 : 우변을 먼저 평가한 후 그 평가값을 좌변에 연결함.
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

# 콜백으로 send() 메소드 사용해야 함
handler = make_handler()
next(handler) #advance to the yield
apply_async(add, (2, 3), callback = handler.send)

[1] Got: 5


In [14]:
apply_async(add, ('hello', 'world'), callback = handler.send)

[2] Got: helloworld


### 3. 추가적인 인자와 partial function 애플리케이션으로
- 해야하는 작업이 추가적인 값을 콜백에 전달하는 것 뿐일때 유용

In [15]:
class SequenceNo:
    def __init__(self):
        self.sequence = 0
        
def handler(result, seq):
    seq.sequence += 1
    print('[{}] Got : {}'.format(seq.sequence, result))
    
seq = SequenceNo()
from functools import partial
apply_async(add, (2, 3), callback = partial(handler, seq=seq))

[1] Got : 5


In [16]:
apply_async(add, ('hello', 'world'), callback = partial(handler, seq=seq))

[2] Got : helloworld


- 콜백 함수가 여러 단계에 걸쳐 실행을 계속하도록 만들기 위해선 어떻게 관련 상태를 저장하고 불러올지 정해야 함

- 상태 고정 저장하는 방법

1) 인스턴스에 상태 저장 (바운드 메소드)

2) 클로저에 저장(내부 함수) -> 더 가볍고 자연스러움 + 코드에서 자동으로 상태 결정

## 8.1 인스턴스의 문자열 표현식 변형
- 결과물 좀 더 보기 좋게 바꾸자

In [51]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self): #객체의 공식적인 문자열 출력 (시스템이 해당 객체를 인식할 수 있는 공식적인 문자열로 나타내줄 때 사용)
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self): #객체의 비공식적인 문자열 출력 (사용자가 보기 쉬운 형태로 qhduwnf Eo)
        return '({0.x!s}, {0.y!s})'.format(self)


- __ repr__() : 인스턴스의 코드 표현식 반환 / 주로 기계가 읽을 수 있는 출력을 생성하기 위한 것
    
- __ str__() : 인스턴스를 문자열로 변환, str()와 print() 함수가 출력되는 결과가 됨. 사람이 읽을 수 있도록 만들어짐

In [18]:
p = Pair(3, 4)
p ## __repr__() 결과

Pair(3, 4)

In [19]:
print(p) #__str__() 결과

(3, 4)


In [20]:
p = Pair(5, 6)
print('p is {0!r}'.format(p))

p is Pair(5, 6)


In [21]:
print('p is {0}'.format(p))

p is (5, 6)


* the goal of __ repr__ is to be unambiguous
- %r 포매팅시 사용되는 스페셜 메서드
- 출력값의 목표는 객체를 구분지어 표현하는 것

- the goal of '__ str__' is to be readable
- 디버깅시 사람이 그 객체를 읽기 편한 값을 출력하도록 정의

## 8.4 인스턴스를 많이 생성할 때 메모리 절약
- adding the __ slots__ attribute to the class definition

__ slots__ 선언은 인스턴스 변수 시퀀스와 각 인스턴스별로 각 변수들에 값을 넣어둘 충분한 공간을 준비해둠.
각 인스턴스별 사전을 만들지 않으므로 공간을 절약할 수 있음

 __ dict__ 나 __ weakref__ 의 생성을 막고 동적으로 선언될 변수들을 위한 공간을 확보

- 해당 클래스에 의해 만들어진 객체의 인스턴스 속성 관리에 딕셔너리 대신 __ slots__ 사용

인스턴스마다 딕셔너리를 구성하지 않고 튜플이나 리스트 같은 부피가 작은 고정 배열로 인스턴스 만들어짐.

In [56]:
class WithoutSlots:
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier

class WithSlots:
    __slots__=['name', 'identifier']
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
    
without_slots = WithoutSlots('PlanB', 1)
print(without_slots.__dict__)
without_slots.new_attr = 3
print(without_slots.__dict__) #새 속성을 설정할 수 있음

{'name': 'PlanB', 'identifier': 1}
{'name': 'PlanB', 'new_attr': 3, 'identifier': 1}


In [58]:
with_slots = WithSlots('PlanA', 2)
print(with_slots.__slots__)
print(with_slots.__dict__)

['name', 'identifier']


AttributeError: 'WithSlots' object has no attribute '__dict__'

In [22]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

* __ dict__ 가 없으므로 인스턴스들은 __ slots__에 적혀진 변수외에는 새로운 변수들을 추가로 만들 수 없음

-> 외부에서 동적으로 지정된 이름 이외의 인스턴스 변수를 만들 수 없도록 제한하려면 __ slots__이용

## 부모 클래스의 메소드 호출
- super() : 자식 클래스에서 부모클래스의 내용을 사용하고 싶을 경우

In [65]:
class A:
    def spam(self):
        print('A.spam')
              
class B(A):
    def spam(self):
        print('B.spam')
        super().spam() #부모의 spam() 호출
B = B()
B.spam() 

B.spam
A.spam


In [73]:
class A:
    def __init__(self):
        self.x = 0
    
class B(A):
    def __init__(self):
        super().__init__() #부모클래스의 __init__ 메소드 호출
        self.y = 1
        
cl = B()
cl.x

0

- super()를 쓰지 않았을 때

In [68]:
class father():
    def handsome(self):
        print('잘생겼다')
class brother(father):
    '''아들'''
class sister(father):
    def pretty(self):
        print('예쁘다')
    def handsome(self):
        '''아빠한테물려받음'''

In [69]:
brother = brother()
brother.handsome()

잘생겼다


In [70]:
sister = sister()
sister.handsome()
sister.pretty()

예쁘다


In [72]:
class sister2(father):
    def pretty(self):
        print('예쁘다')
    def handsome(self):
        super().handsome()
sister2 = sister2()
sister2.handsome()
sister2.pretty()

잘생겼다
예쁘다


In [26]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
    
    def __getattr__(self, name):
        return getattr(self._obj, name)
    
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            setattr(self._obj, name, value)

- super() 적절한 사용

In [28]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')
        
class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')

In [29]:
c = C()

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


In [32]:
class Base:
    def __init__(self):
        print('Base.__init__')
        
class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')
        
class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')
        
class C(A,B):
    def __init__(self):
        super().__init__()     # Only one call to super() here
        print('C.__init__')

In [33]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


In [34]:
# MRO 리스트 : 모든 베이스 클래스를 순차적으로 나열한 리스트
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

## 8.10 Using Lazily Computed Properties
- 읽기 전용 속성을 property로 정의하고 이 속성에 접근할 때만 계산하고 싶음
- 한 번 접근하면 캐시로 놓고 다음 번 접근할 때는 다시 계산 안하고싶음

In [35]:
# 디스크립터 클래스 사용
class lazyproperty:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [81]:
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty    
    def area(self):
        print ('Computing area')
        return math.pi * self.radius ** 2
    
    @lazyproperty
    def perimeter(self):
        print ('Computing perimeter')
        return 2 * math.pi * self.radius

In [82]:
c = Circle(4.0)
c.radius

4.0

In [83]:
c.area

Computing area


50.26548245743669

In [84]:
c.perimeter

Computing perimeter


25.132741228718345

In [85]:
c.perimeter

25.132741228718345

In [88]:
a = Circle(2.0)

In [89]:
#인스턴스 변수 구하기
vars(a)

{'radius': 2.0}

In [90]:
# 면적 계산하고 추후 변수 확인
a.area

Computing area


12.566370614359172

In [91]:
vars(a)

{'area': 12.566370614359172, 'radius': 2.0}

In [92]:
a.area

12.566370614359172

In [93]:
#변수를 삭제하면 다시 프로퍼티 실행됨
del a.area
vars(a)

{'radius': 2.0}

In [94]:
a.area

Computing area


12.566370614359172

In [95]:
# 계산한 값을 생성한 후에 수정할 수 있음
a.area = 25
a.area

25

## 8.13 데이터 모델 혹은 타입 시스템 구현
- 여러 종류의 자료 구조 정의 (특정 값에 제약을 걸어 원하는 속성이 할당되도록)
- descriptor
- mixin classes
- super()
- class decorators
- metaclasses

Descriptor는 특성 객체의 속성 접근 시 먼저 그 속성의 특성을 체크하여 처리할 수 있는 방법
- 구현 class에 대한 getter/setter/deleter 함수를 별도 관리
- 구현시 메소드 정의 없이 변수로 처리로 대처
- descriptor를 사용하면 한 클래스의 서로 다른 많은 속성에 같은 로직을 재사용할 수 있음

Descriptor class를 생성하여 실제 구현 클래스 내부의 속성에 대한 init/setter를 통제할 수 있도록 구조화

실제 인스턴스의 속성에 접근할 때 자동으로 메소드 처리되므로 속성에 접근을 제어하는 기능을 제시할 수 있음

In [37]:
class t(object):
    def __get__(*args, **kwargs):
        print('get')
 
    def __set__(*args, **kwargs):
        print(**kwargs)
        print('set')
        
class test():
    a = t()
    b = t()
    
a = test()
a.a = 10
a.b = 20


set

set


In [29]:
class Grade:
    def __get__(*args, **kwargs):
        pass
    def __set__(*args, **kwargs):
        pass
#Grade class는 디스크립터 프로토콜 구현

In [30]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [31]:
exam = Exam()
exam.writing_grade = 40
# Exam.__dict__['writing_grade'].__set__(exam, 40)

In [4]:
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

- __ set__() 메소드는 존재 but __ get__()없음
-> 디스크립터가 인스턴스 딕셔너리에서 동일한 이름의 값을 추출하는 것 이외에 다른 동작을 하지 않는다면 __ get__()을 정의할 필요가 없음(오히려 속도가 느려진다)

In [5]:
class A:
    name = Descriptor
    
a = A()
a.name = 'Dahl'
print(a.name)

Dahl


In [6]:
#타입을 강제하기 위한 디시크립터
# 서브클래스가 제공한 expected_type 속성을 찾기만 함
class Typed(Descriptor):
    expected_type = type(None)
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected' + str(self.expected_type))
        super().__set__(instance, value)

In [7]:
#값을 강제하기 위한 디스크립터
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0 :
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)
        

In [8]:
class MaxSized(Descriptor):
    
    ##디스크립터의 모든 __init__() 메소드는 키워드 매개변소 **opts를 포함해서 동일한 시그니처를 가지도록 프로그램 되어 있음
    def __init__(self, name = None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)
        
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be <' + str(self.size))
        super().__set__(instance, value)

In [9]:
class Integer(Typed):
    expected_type = int
    
class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float
    
class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str
    
class SizedString(String, MaxSized):
    pass

In [10]:
class Stock:
    name = SizedString('name', size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [12]:
s = Stock('ACME', 50, 91.1)
s.name
s.shares
s.price

91.1

- stock 에 값을 할당할 때 주어진 제약으로 검증이 이루어짐

In [19]:
s.shares = 75

In [20]:
s.shares = -10

ValueError: Expected >= 0

In [21]:
s.price = 'a lot'

TypeError: expected<class 'float'>

In [22]:
s.name = 'ABBRCDDAAD'

ValueError: size must be <8

### 클래스 데코레이터(decorator) 사용 
- 가장 유연하고 안전함

In [40]:
import datetime

def main_function():
    print (datetime.datetime.now())
    print ('MAIN FUNCTION START')
    print (datetime.datetime.now())

-decorator 사용

In [41]:
def datetime_decorator(func):
    def decorated():
        print (datetime.datetime.now())
        func()
        print (datetime.datetime.now())
    return decorated

In [42]:
@datetime_decorator
def main_function_1():
    print ('MAIN FUNCTION1 START')
    
@datetime_decorator
def main_function_2():
    print ('MAIN FUNCTION2 START')
    
@datetime_decorator
def main_function_3():
    print ('MAIN FUNCTION3 START')

In [24]:
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decorate

In [25]:
@check_attributes(name = SizedString(size=8),
                 shares = UnsignedInteger,
                 price = UnsignedFloat)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

### 메타클래스
- 메타클래스는 객체를 만드는 '무언가'
- 메타클래스는 '클래스'의 '클래스'

In [26]:
# 확인을 위한 메타클래스
class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                values.name = key
        return type.__new__(cls, clsname, bases, methods)

In [ ]:
class Stock(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

- 클래스 데코레이터 방식은 mixin class, 다중 상속, 복잡한 super() 대신 사용 가능

__ call__ 함수로 decorator 형식 정의

In [14]:
import datetime

class DatetimeDecorator:
    def __init__(self, f):
        self.func = f
        
    # decorator 클래스가 적용된 함수의 parameter를 *args, **kwargs 가변인자로 받음
    def __call__(self, *args, **kwargs):
        print (datetime.datetime.now())
        self.func(*args, **kwargs)
        print (datetime.datetime.now())

In [15]:
class MainClass:
    @DatetimeDecorator
    def main_func_1():
        print ('MAIN FUNCTION 1 START')
        
    @DatetimeDecorator
    def main_func_2():
        print ('MAIN FUNCTION 2 START')
        
    @DatetimeDecorator
    def main_func_3():
        print ('MAIN FUNCTION 3 START')
        
MY = MainClass()
MY.main_func_1()


2019-05-05 07:59:31.538028
MAIN FUNCTION 1 START
2019-05-05 07:59:31.538238


In [13]:
# 베이스 클래스 (값을 설정할 때 디스크립터 사용)
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
            
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

In [16]:
# 타입 확인에 데코레이터 사용
def Typed(expected_type, cls=None):
    if cls is None:
        return lambda cls: Typed(expected_type, cls)
    
    super_set = cls.__set__
    def __set__(self, instance, value):
        if not isinstance(value, expected_type):
            raise TypeError('expected '+str(expected_type))
        super_set(self, instance, value)
        
    cls.__set__ = __set__
    return cls

#unsigned 값에 데코레이터 사용
def Unsigned(cls):
    super_set = cls.__set__
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

#크기 있는 값에 데코레이터 사용
def MaxSized(cls):
    super_init = cls.__init__
    def __init__(self, name = None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super_init(self, name, **opts)
    cls.__init__ = __init__
    
    super_set = cls.__set__
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

In [17]:
# 특별 디스크립터
@Typed(int)
class Integer(Descriptor):
    pass

@Unsigned
class UnsignedInteger(Integer):
    pass

@Typed(float)
class Float(Descriptor):
    pass

@Unsigned
class UnsignedFloat(Float):
    pass

@Typed(str)
class String(Descriptor):
    pass

@MaxSized
class SizedString(String):
    pass
    

* 위의 데코레이터와 완전히 동일하게 동작 + 실행 속도 훨씬 빠름

## 8.17 init 호출 없이 인스턴스 생성
- class의 __ new__() 메소드 호출

In [45]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

In [48]:
d = Date.__new__(Date)
d

In [1]:
d.year ##초기화되지는 않음

NameError: name 'd' is not defined

적절한 인스턴스 변수를 설정하는 것은 사람이 알아서

In [50]:
data = {'year':2012, 
       'month':8,
       'day':29}
for key, value in data.items():
    setattr(d, key, value)
    
d.year

2012

In [51]:
d.month

8

__ init__()을 생략하면 비표준 방식으로 인스턴스를 생성할 때 문제 발생(데이터 역직렬화, 대안 생성자로 정의한 클래스)

## 8.21 Implementing the Visitor Pattern
- 여러 종류의 객체로 구성된 복잡한 자료 구조를 순환하며 서로 다른 방식으로 처리하는 ㅋ코드를 작성

In [25]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand
        
class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
class Add(BinaryOperator):
    pass 

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

In [28]:
t1 = Sub(Number(3), Number(4)) # 3 - 4
t2 = Mul(Number(2), t1) # 2*(3-4)
t3 = Div(t2, Number(5)) # 2*(3-4)/5
t4 = Add(Number(1), t3)
# 1 + 2 * (3 - 4) / 5
# 중첩된 자료 구조

In [29]:
class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))
        
# 모든 데이터 취급

In [30]:
# NodeVisitor 상속받음.
# visit_name 형식의 메소드 구현
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right) #재귀적 호출
    
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    
    def visit_Negate(self, node):
        return -node.operand

In [31]:
e = Evaluator()
e.visit(t4)

0.6

## 8.25  캐시 인스턴스 생성

weak reference (약한 참조) :
    레퍼런스 카운트로 고려되지 않는 레퍼런스
    
- weakref
- weakref.ref() - 약한 참조 객체 생성
- weakref.proxy() - 프록시 객체 생성

weak dictionary (cache) : key or value로 사용되는 객체는 약한 참조가 가능해야 함
    
    객체가 삭제되면 자동적으로 캐쉬에 있는 key value 쌍도 삭제됨

In [16]:
import weakref
class C :
    pass
c = C()
c.a = 4
d = weakref.WeakValueDictionary()
d[1] = c # 항목 생성
d.items() # 사전 내용 확인

<generator object WeakValueDictionary.items at 0x7f13344e01a8>

In [17]:
d[1].a # 참조

4

In [12]:
del c #객체 삭제
d.items()

<generator object WeakValueDictionary.items at 0x7f13344e0048>

In [ ]:
class Spam:
    def __init__(self, name):
        self.name = name

In [4]:
# 캐시 지원
import weakref
#Weak dictionary는 구성요소인 키(Key)와 값(Value)중 하나가 약한 참조로 이루어져 있는 경우
#각 개체에 ID를 키로 부여하여 사전에 넣어서 관리하는 경우, 사전에 넣는 순간 벌써 참조카운터가 하나 증가함
#반면에 약한 사전의 경우 사용자가 사전에 입력을 해도 참조카운터가 증가되지 않으며, 원본객체가 삭제되면 자동적으로 사전의 객체도 삭제되어 None이 된다.

_spam_cache = weakref.WeakValueDictionary() # value를 약한 참조로 갖는 사전객체를 생성

def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else :
        s = _spam_cache[name]
    return s

In [5]:
a = get_spam('foo')
b = get_spam('bar')
a is b

False

In [6]:
c = get_spam('foo')
a is c

True


weakvaluedictionary 인스턴스는 참조한 아이템이 존재하는 경우에만.

인스턴스가 없으면 딕셔너리 키가 사라짐

In [13]:
a = get_spam('foo')
b = get_spam('bar')
c = get_spam('foo')
list(_spam_cache)

['bar', 'foo']

In [14]:
del a
del c
list(_spam_cache)

['bar']

In [15]:
del b
list(_spam_cache)

[]

In [19]:
import weakref
class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()
    def get_spam(self, name):
        if name not in self._cache:
            s = Spam(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s
    
    def clear(self):
        self._cache.clear()

In [20]:
class Spam:
    manager = CachedSpamManager()
    def __init__(self, name):
        self.name = name

def get_spam(name):
    return Spam.manager.get_spam(name)

별도의 클래스로 서로 다른 관리 스킴을 구현하고 Spam 클래스에 붙여서 기본 캐싱 구현으로 치환할 수 있음

In [21]:
a = Spam('foo')
b = Spam('foo')
a is b

False